### Imports

In [1]:

import networkx as nx
import numpy as np
import random, math
import graphviz
import functools, collections, operator
import seaborn as sns

from scipy.stats import shapiro
import scipy.stats as scs
from scipy.stats import describe

import plotly.figure_factory as ff

from scipy.stats import norm
import scipy.stats as stats
from scipy.stats import pearsonr

import os
import time

import pylab

from statsmodels.graphics.gofplots import qqplot

#import kaleido

import pandas as pd


import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm

import plotly.graph_objects as go
from pyvis.network import Network
from networkx.drawing.nx_agraph import graphviz_layout
import plotly.express as px
from plotly._subplots import make_subplots
import plotly

import statsmodels.formula.api as sm

from stargazer.stargazer import Stargazer


# import models
from classes.jackson_model import JacksonSimulationV2
from classes.network_drawing import plotly_sim_drawing
from classes.plotly_drawings import plotly_lines


def MTO_simulator(initial_n=0, T=100, m=10, pm_o=1, n=10, pn_o=1, p_SES_high=0.5, list_rho_values=[1], list_eps_values=[1], num_sim_per_param=1, num_MTO_per_sim=10):

    total_sims = 0
    list_sim_stats = []
    n_sims = len(list_eps_values) * len(list_rho_values) * num_sim_per_param * num_MTO_per_sim
    t0 = time.time()

    print(f'Running {n_sims} simulations over {len(list_eps_values) * len(list_rho_values)} parameter combinations')

    # iterate over eps values
    for i in range(len(list_eps_values)):

        # iterate over rho values
        for j in range(len(list_rho_values)):

            # create a network and run MTO sims
            for k in range(num_sim_per_param):
                # create network
                network_sim = JacksonSimulationV2(initial_n, T, m, pm_o, n, pn_o, p_SES_high, biased=True, epsilon = list_eps_values[i], rho=list_rho_values[j])

                # run MTO simulations
                df_MTO_sim = network_sim.MTO_sim_many(network_sim.graph_history[-1], n_sims=num_MTO_per_sim)


                # calculate network-wide characteristics
                SES_assortativity = nx.assortativity.attribute_assortativity_coefficient(network_sim.graph_history[-1], 'SES_High')
                H_Share_H, H_Share_L = network_sim.helper_functions().average_neighbour_type_per_SES(network_sim.graph_history[-1])
                Network_Degree = sum(dict(network_sim.graph_history[-1].degree()).values()) / len(network_sim.graph_history[-1])
                Network_Clustering = nx.average_clustering(network_sim.graph_history[-1])

                list_network_chars = [SES_assortativity, H_Share_H, H_Share_L, Network_Degree, Network_Clustering]
                list_network_char_names = ['Network_SES_assortativity', 'Network_H_Share_H', 'Network_H_Share_L', 'Network_Degree', 'Network_Clustering']

                # add the values to the df
                for n in range(len(list_network_chars)):
                    df_MTO_sim[list_network_char_names[n]] = list_network_chars[n]

                # append to list
                list_sim_stats.append(df_MTO_sim)
                print('\r', f'Finished simulation {k}/{num_sim_per_param} for combination {total_sims}/{len(list_eps_values)*len(list_rho_values)}', end='')
        
            total_sims += 1

    delta_t = round((time.time()-t0)/60, 2)

    print('\r', f'Finished {n_sims} simulations in {delta_t} minutes, average time: {delta_t/n_sims} min/sim', end='')
    df_MTO_output = pd.concat(list_sim_stats).reset_index(drop=True)

    # add predicted values
    df_MTO_output['Predicted_H_Share'] = 1/((1+df_MTO_output['epsilon'])*(df_MTO_output['rho']))
    df_MTO_output['Predicted_Exposure'] = 1/(df_MTO_output['p_SES_high']*(1+df_MTO_output['epsilon']))
    df_MTO_output['Predicted_Friend_Bias'] = (df_MTO_output['epsilon']*(df_MTO_output['rho']-1))/(df_MTO_output['epsilon']*df_MTO_output['rho']+1)


    return df_MTO_output


## Showcases

### Showcases

In [12]:

# showcases
showcase_sim_no_bias = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=False, rho=1, epsilon=1)
showcase_sim_expo = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=True, rho=1, epsilon=1.75)
showcase_sim_fb = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=True, rho=1.75, epsilon=1)
showcase_sim_expo_fb = JacksonSimulationV2(0, 100, 10, 1, 10, 1, 0.5, biased=True, rho=1.5, epsilon=1.5)

showcase_sims = [showcase_sim_no_bias, showcase_sim_expo, showcase_sim_fb, showcase_sim_expo_fb]

showcase_plots = [plotly_sim_drawing.plotly_draw(plotly_sim_drawing(), showcase_sim, layout='spring', draw_largest_CC=True, title=None) for showcase_sim in showcase_sims]





### Rho and Epsilon distributions

In [3]:

# open Chetty data
df_exp_fe_chetty = pd.read_csv('EpsRhoCalcs.csv')[['county', 'county_name', 'exposure_grp_mem_county', 'exposure_grp_mem_high_county', 'bias_grp_mem_county', 'bias_grp_mem_high_county']].dropna()
df_exp_fe_chetty = df_exp_fe_chetty.apply(pd.to_numeric, errors='ignore')


# calculate epsilon based on high and low exposure values
df_exp_fe_chetty['Epsilon'] = 1/(0.5*df_exp_fe_chetty['exposure_grp_mem_county']) - 1
#df_exp_fe_chetty['EpsilonHigh'] = 0.5*df_exp_fe_chetty['exposure_grp_mem_high_county']/(1 - 0.5*df_exp_fe_chetty['exposure_grp_mem_high_county'])

# calculate rho values: how to approach this?
df_exp_fe_chetty['Rho'] = (df_exp_fe_chetty['Epsilon'] + df_exp_fe_chetty['bias_grp_mem_county'])/(df_exp_fe_chetty['Epsilon']*(1 - df_exp_fe_chetty['bias_grp_mem_county']))
#df_exp_fe_chetty['RhoHigh'] = (1 - df_exp_fe_chetty['bias_grp_mem_high_county'])/(df_exp_fe_chetty['bias_grp_mem_high_county'] * df_exp_fe_chetty['EpsilonHigh'] + 1)

df_eps_rho_chetty = df_exp_fe_chetty.copy()

df_eps_rho_chetty.describe()
df_eps_rho_chetty.var()


C:\Users\rasmu\AppData\Local\Temp\ipykernel_3320\3350318442.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_eps_rho_chetty.var()


county                          2.292003e+08
exposure_grp_mem_county         4.486528e-02
exposure_grp_mem_high_county    4.391154e-02
bias_grp_mem_county             2.563738e-03
bias_grp_mem_high_county        4.178828e-03
Epsilon                         4.562847e-01
Rho                             1.289756e-02
dtype: float64

In [171]:

'''DESCRIPTIVE STATISTICS'''

print(pearsonr(df_eps_rho_chetty['Epsilon'], df_eps_rho_chetty['Rho']))


np.corrcoef(df_eps_rho_chetty['Epsilon'], df_eps_rho_chetty['Rho'])

eps_more_than_one = len(df_eps_rho_chetty[df_eps_rho_chetty['Epsilon'] >= 1]) / len(df_eps_rho_chetty)
rho_more_than_one = len(df_eps_rho_chetty[df_eps_rho_chetty['Rho'] >= 1]) / len(df_eps_rho_chetty)


df_chetty_summ = df_eps_rho_chetty[['Epsilon', 'Rho']].describe().drop('count')
df_chetty_summ.loc['skew'] = df_eps_rho_chetty.skew(numeric_only=True)
df_chetty_summ.loc['%>1'] = {'Epsilon': eps_more_than_one, 'Rho': rho_more_than_one}

df_chetty_summ = df_chetty_summ.round(4)
df_chetty_summ.to_latex('data.tex')

df_chetty_summ


PearsonRResult(statistic=-0.010843192139434766, pvalue=0.5519349234302265)


C:\Users\rasmu\AppData\Local\Temp\ipykernel_2816\3712884401.py:17: FutureWarning:

In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.



,Epsilon,Rho
mean,1.3546,1.1316
std,0.6755,0.1136
min,0.3456,0.7967
25%,0.8999,1.0595
50%,1.2082,1.1255
75%,1.6126,1.1929
max,6.8370,2.5296
skew,1.8287,1.2903
%>1,0.6630,0.9017


In [3]:

'''FIGURES'''

# import the data, convert
#df_eps_rho_chetty = pd.read_csv('EpsRhoCalcs.csv')[['county', 'county_name', 'exposure_grp_mem_county', 'exposure_grp_mem_high_county', 'bias_grp_mem_county', 'bias_grp_mem_high_county', 'Epsilon', 'EpsilonHigh', 'Rho', 'RhoHigh']].dropna()
#df_eps_rho_chetty = df_eps_rho_chetty.apply(pd.to_numeric, errors='ignore')


vars = ['Epsilon', 'Rho']# 'EpsilonHigh', 'Rho', 'RhoHigh']
figs = []
titles = []

for var in vars:
    # Winsorize at the 0.5% level
    qabove = df_eps_rho_chetty[var].quantile(0.005)
    qbelow = df_eps_rho_chetty[var].quantile(0.995)
    print(qbelow, qabove)

    df_eps_rho_chetty_var = df_eps_rho_chetty[(df_eps_rho_chetty[var] >= df_eps_rho_chetty[var].quantile(0.005)) & (df_eps_rho_chetty[var] <= df_eps_rho_chetty[var].quantile(0.995))]


    #calculate mean and variance
    mu, std = norm.fit(df_eps_rho_chetty_var[var])

    # Shapiro Wilk test (want p<0.05)
    stat, p = shapiro(df_eps_rho_chetty_var[var])

    # count share below 1
    below_one_share = len([x for x in list(df_eps_rho_chetty_var[var]) if x < 1]) / len(list(df_eps_rho_chetty_var[var]))

    fig_title = f'{var} density plot<br>Winsorised at 0.5% level<br>Normal distribution: mean={round(mu,3)}, se={round(std, 3)}<br>Shapiro-Wilk stat and prob: {format(stat, ".3f")}, {"{:.1e}".format(p)}<br>Prob <1: {round(below_one_share, 3)}'

    # Create a histogram of the column
    fig = px.histogram(df_eps_rho_chetty, x=var, nbins=100, opacity=0.9, marginal=None, histnorm='probability density', width=600, height=600)#, color=df_eps_rho_chetty[var + 'Color'])

    # Add a normal distribution curve to the histogram
    x_axis = np.linspace(df_eps_rho_chetty[var].min(), df_eps_rho_chetty[var].max(), 100)
    y_axis = norm.pdf(x_axis, mu, std)
    curve = go.Scatter(x=x_axis, y=y_axis, mode='lines', line=dict(width=2, color='black'), name=f'Normal Distribution: ', showlegend=False)
    fig.add_trace(curve)

    figs.append(fig)
    titles.append(fig_title)

for var in vars:
    stat, p = shapiro(df_eps_rho_chetty[var])
    print(f'{var} has a SW stat of {format(stat, ".3f")} with p-value {"{:.1e}".format(p)}')

fig_eps_dist, fig_rho_dist = figs



# improve the exp distribution plot
fig_eps_dist.update_layout(    
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Probability Density',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.65),
    legend_title_text = '',
    xaxis_range = [0.25,4],
    yaxis_range = [0,1],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')

fig_eps_dist.update_traces(marker=dict(color='rgb(0,200,50)'), selector=dict(type='histogram'))
fig_eps_dist.update_traces(line=dict(color='rgb(6, 52, 17)'), selector=dict(type='scatter'), line_width = 4)


eps_median = df_eps_rho_chetty['Epsilon'].median()

fig_eps_dist.add_vline(x=1, line=dict(color='black'), annotation_text = 'No bias ', annotation_position='top left', line_width=3, annotation_font_color='black')
fig_eps_dist.add_vline(x=eps_median, line=dict(color='black'), line_dash='dash', annotation_position='top right', line_width=2, annotation_text = f' Median: {round(eps_median, 2)}', annotation_font_color='black')


# improve the rho distribution plot
fig_rho_dist.update_layout(    
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'Probability Density',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.65),
    legend_title_text = '',
    xaxis_range = [0.75,2],
    yaxis_range = [0,5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')

fig_rho_dist.update_traces(marker=dict(color='rgb(200,0,200)'), selector=dict(type='histogram'))
fig_rho_dist.update_traces(line=dict(color='rgb(103, 12, 103)'), selector=dict(type='scatter'), line_width = 4)


rho_median = df_eps_rho_chetty['Rho'].median()

fig_rho_dist.add_vline(x=1, line=dict(color='black'), annotation_text = 'No bias ', annotation_position='top left', line_width=3, annotation_font_color='black')
fig_rho_dist.add_vline(x=rho_median, line=dict(color='black'), line_dash='dash', annotation_position='top right', line_width=2, annotation_text = f' Median: {round(rho_median, 2)}', annotation_font_color='black')


3.9505651121729164 0.4300924001831513
1.5409958225597886 0.8707689351122104
Epsilon has a SW stat of 0.866 with p-value 4.2e-45
Rho has a SW stat of 0.945 with p-value 4.8e-32


### General simulation code

In [3]:

# runs a lot of MTO simulations for a givne list of epsilon and rho values, returning a df
def MTO_simulator(initial_n=0, T=100, m=10, pm_o=1, n=10, pn_o=1, p_SES_high=0.5, list_rho_values=[1], list_eps_values=[1], num_sim_per_param=1, num_MTO_per_sim=10):

    total_sims = 0
    list_sim_stats = []
    n_sims = len(list_eps_values) * len(list_rho_values) * num_sim_per_param * num_MTO_per_sim
    t0 = time.time()

    print(f'Running {n_sims} simulations over {len(list_eps_values) * len(list_rho_values)} parameter combinations')

    # iterate over eps values
    for i in range(len(list_eps_values)):

        # iterate over rho values
        for j in range(len(list_rho_values)):

            # create a network and run MTO sims
            for k in range(num_sim_per_param):
                # create network
                network_sim = JacksonSimulationV2(initial_n, T, m, pm_o, n, pn_o, p_SES_high, biased=True, epsilon = list_eps_values[i], rho=list_rho_values[j])

                # run MTO simulations
                df_MTO_sim = network_sim.MTO_sim_many(network_sim.graph_history[-1], n_sims=num_MTO_per_sim)


                # calculate network-wide characteristics
                SES_assortativity = nx.assortativity.attribute_assortativity_coefficient(network_sim.graph_history[-1], 'SES_High')
                H_Share_H, H_Share_L = network_sim.helper_functions().average_neighbour_type_per_SES(network_sim.graph_history[-1])
                Network_Degree = sum(dict(network_sim.graph_history[-1].degree()).values()) / len(network_sim.graph_history[-1])
                Network_Clustering = nx.average_clustering(network_sim.graph_history[-1])

                list_network_chars = [SES_assortativity, H_Share_H, H_Share_L, Network_Degree, Network_Clustering]
                list_network_char_names = ['Network_SES_assortativity', 'Network_H_Share_H', 'Network_H_Share_L', 'Network_Degree', 'Network_Clustering']

                # add the values to the df
                for n in range(len(list_network_chars)):
                    df_MTO_sim[list_network_char_names[n]] = list_network_chars[n]

                # append to list
                list_sim_stats.append(df_MTO_sim)
                print('\r', f'Finished simulation {k}/{num_sim_per_param} for combination {total_sims}/{len(list_eps_values)*len(list_rho_values)}', end='')
        
            total_sims += 1

    delta_t = round((time.time()-t0)/60, 2)

    print('\r', f'Finished {n_sims} simulations in {delta_t} minutes, average time: {delta_t/n_sims} min/sim', end='')
    df_MTO_output = pd.concat(list_sim_stats).reset_index(drop=True)

    # add predicted values
    df_MTO_output['Predicted_H_Share'] = 1/((1+df_MTO_output['epsilon'])*(df_MTO_output['rho']))
    df_MTO_output['Predicted_Exposure'] = 1/(df_MTO_output['p_SES_high']*(1+df_MTO_output['epsilon']))
    df_MTO_output['Predicted_Friend_Bias'] = (df_MTO_output['epsilon']*(df_MTO_output['rho']-1))/(df_MTO_output['epsilon']*df_MTO_output['rho']+1)


    return df_MTO_output


#df_test = MTO_simulator(list_rho_values=[1,2], list_eps_values=[1,2], num_MTO_per_sim=100, num_sim_per_param=5)
#df_test.sample(5)


## Epsilon

### Epsilon code

In [2]:

# general simulation stuff
list_eps_values = np.arange(0.5, 2, 0.01)

num_sim_per_param = 10 # number of simulations with given parameter set
num_MTO_per_sim = 100 # number of simulations on each network


#df_eps = MTO_simulator(list_rho_values=[1, 1.25, 1.5], list_eps_values=list_eps_values, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_eps.sample(5)
#df_eps.to_csv('eps_sim.csv')

df_eps = pd.read_csv('SimulationData/eps_sim_new.csv').drop(columns='Unnamed: 0')
df_eps.sample(5)

#df_eps = df_eps.rename(columns={'Network_H_Share_H': 'H_Share_H', 'Network_H_Share_L': 'H_Share_L', 'Predicted_H_Share': 'Predicted_H_Share_L'})
#df_eps['Predicted_H_Share_H'] = 1-df_eps['Predicted_H_Share_L']

df_eps


,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,Network_H_Share_H,Network_H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias
0,wFtrawGg,0.5,0.50,1.0,0.550000,1.100000,0.000000,20,20,-0.263615,0.330395,0.608020,31.62,0.359835,0.666667,1.333333,0.000000
1,wFtrawGg,0.5,0.50,1.0,0.700000,1.400000,0.000000,20,20,-0.263615,0.330395,0.608020,31.62,0.359835,0.666667,1.333333,0.000000
2,wFtrawGg,0.5,0.50,1.0,0.736842,1.473684,0.000000,20,19,-0.263615,0.330395,0.608020,31.62,0.359835,0.666667,1.333333,0.000000
3,wFtrawGg,0.5,0.50,1.0,0.647059,1.294118,0.000000,20,17,-0.263615,0.330395,0.608020,31.62,0.359835,0.666667,1.333333,0.000000
4,wFtrawGg,0.5,0.50,1.0,0.555556,1.111111,0.000000,20,18,-0.263615,0.330395,0.608020,31.62,0.359835,0.666667,1.333333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449995,eMfPqqoT,0.5,1.99,1.5,0.583333,1.285714,0.092593,14,12,0.415076,0.777134,0.345408,21.60,0.279903,0.222965,0.668896,0.249686
449996,eMfPqqoT,0.5,1.99,1.5,0.428571,0.857143,0.000000,14,14,0.415076,0.777134,0.345408,21.60,0.279903,0.222965,0.668896,0.249686
449997,eMfPqqoT,0.5,1.99,1.5,0.333333,0.857143,0.222222,14,12,0.415076,0.777134,0.345408,21.60,0.279903,0.222965,0.668896,0.249686
449998,eMfPqqoT,0.5,1.99,1.5,0.285714,0.571429,0.000000,14,14,0.415076,0.777134,0.345408,21.60,0.279903,0.222965,0.668896,0.249686


In [86]:
df_eps = df_eps.rename(columns={'Network_H_Share_H': 'H_Share_H', 'Network_H_Share_L': 'H_Share_L', 'Predicted_H_Share': 'Predicted_H_Share_L'})
df_eps['Predicted_H_Share_H'] = 1-df_eps['Predicted_H_Share_L']

### Epsilon HSES Share

In [99]:



HSESL_epsilon_rho125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.25], x_var='epsilon', y_var='H_Share_L', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=True, colors=['rgba(0,0,200,1)', 'rgba(0,0,200,1)'], legend_text='Low')

HSESH_epsilon_rho125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.25], x_var='epsilon', y_var='H_Share_H', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=True, colors=['rgba(200,0,0,1)', 'rgba(200,0,0,1)'], legend_text='High')

HSESL_epsilon_rho125.data[3]['line']['dash'] = 'dash'
HSESH_epsilon_rho125.data[3]['line']['dash'] = 'dash'

fig_HSES_epsilon = go.Figure(data = HSESL_epsilon_rho125.data + HSESH_epsilon_rho125.data)

fig_HSES_epsilon.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'HSES Share',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.5),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.25,0.75],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_HSES_epsilon.add_hline(y=0.5, line_width=2, annotation_text = '', annotation_position='top left', annotation_font_color='black', line=dict(color='black'))
fig_HSES_epsilon.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no bias', annotation_font_color='black', line=dict(color='black'))


fig_HSES_epsilon.data[0]['name'] = 'Realised  L'
fig_HSES_epsilon.data[3]['name'] = 'Predicted L'
fig_HSES_epsilon.data[6]['name'] = 'Realised  H'
fig_HSES_epsilon.data[9]['name'] = 'Predicted H'

fig_HSES_epsilon


### Epsilon Assortativity

In [49]:


SESAss_epsilon_rho1 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1], x_var='epsilon', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(250,0,250,1)'], legend_text='ρ=1')

SESAss_epsilon_rho125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.25], x_var='epsilon', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(150,0,150,1)'], legend_text='ρ=1.25')

SESAss_epsilon_rho15 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.5], x_var='epsilon', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=False, colors=['rgba(100,0,100,1)'], legend_text='ρ=1.5')

SESAss_epsilon_rho = go.Figure(data = SESAss_epsilon_rho15.data + SESAss_epsilon_rho125.data + SESAss_epsilon_rho1.data)

SESAss_epsilon_rho.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'SES Assortativity',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.15),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [-0.5,0.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')

SESAss_epsilon_rho.add_hline(y=0, line_width=2, annotation_text = 'Random sorting', annotation_font_color='black', line=dict(color='black'))
SESAss_epsilon_rho.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no bias', annotation_font_color='black', line=dict(color='black'))




### Epsilon Exposure DONE

In [103]:


Exposure_epsilon_rho125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.25], x_var='epsilon', y_var='Exposure', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=True, colors=['rgba(15,100,84,1)', 'rgba(15,100,84,1)'])

fig_Exposure_epsilon = go.Figure(data =Exposure_epsilon_rho125.data)

fig_Exposure_epsilon.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Exposure',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.95),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,1.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')

fig_Exposure_epsilon.data[3]['line']['dash'] = 'dash'

fig_Exposure_epsilon.add_hline(y=1, line_width=2, annotation_text = 'Even exposure', annotation_font_color='black', line=dict(color='black'))
fig_Exposure_epsilon.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no bias', annotation_font_color='black', line=dict(color='black'))


### Epsilon Friending bias DONE

In [104]:

Friending_epsilon_rho125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_eps[df_eps['rho'] == 1.25], x_var='epsilon', y_var='Friend_Bias', x_range=[0.5,2], y_range=[0,1], x_title='Epsilon', y_title='Exposure', with_predicted=True, colors=['rgba(229,187,0,1)', 'rgba(224,116,0,1)'])

fig_Friending_epsilon = go.Figure(data = Friending_epsilon_rho125.data)

fig_Friending_epsilon.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Friending Effect',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.85),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0,1],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')

fig_Friending_epsilon.data[3]['line']['dash'] = 'dash'

fig_Friending_epsilon.add_hline(y=0.5, line_width=2, annotation_text = 'No preference', annotation_font_color='black', line=dict(color='black'))
fig_Friending_epsilon.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no bias', annotation_font_color='black', line=dict(color='black'))


## Rho

### Rho code

In [88]:

# general simulation stuff
list_rho_values = np.arange(0.5, 2, 0.01)


num_sim_per_param = 10 # number of simulations with given parameter set
num_MTO_per_sim = 100 # number of simulations on each network


#df_rho = MTO_simulator(list_eps_values=[1, 1.25, 1.5], list_rho_values=list_rho_values, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_rho.to_csv('rho_sim_new.csv')

df_rho = pd.read_csv('SimulationData/rho_sim_new.csv').drop(columns='Unnamed: 0')

df_rho = df_rho.rename(columns={'Network_H_Share_H': 'H_Share_H', 'Network_H_Share_L': 'H_Share_L', 'Predicted_H_Share': 'Predicted_H_Share_L'})
df_rho['Predicted_H_Share_H'] = 1-df_rho['Predicted_H_Share_L']

df_rho.sample(5)


,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,H_Share_H,H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share_L,Predicted_Exposure,Predicted_Friend_Bias,Predicted_H_Share_H
436299,ODGXuaRo,0.5,1.5,1.86,0.200000,0.769231,0.480,14,10,0.426579,0.733215,0.287357,19.62,0.266810,0.215054,0.8,0.340369,0.784946
311021,klouQgXU,0.5,1.5,0.61,0.357143,0.714286,0.000,14,14,0.123959,0.483495,0.350838,24.26,0.307294,0.655738,0.8,-0.305483,0.344262
422267,AnVRzZPC,0.5,1.5,1.72,0.500000,1.142857,0.125,14,12,0.374461,0.712004,0.304488,20.86,0.278122,0.232558,0.8,0.301676,0.767442
40929,FsiTvMKW,0.5,1.0,0.90,0.538462,1.076923,0.000,14,13,-0.027037,0.500665,0.522601,24.52,0.301422,0.555556,1.0,-0.052632,0.444444
364658,vOSNIbWt,0.5,1.5,1.14,0.500000,1.000000,0.000,14,14,0.183966,0.535988,0.341902,23.20,0.279741,0.350877,0.8,0.077491,0.649123


### Rho HSES Share

In [105]:



HSESL_rho_epsilon125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.25], x_var='rho', y_var='H_Share_H', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=True, colors=['rgba(200,0,0,1)', 'rgba(200,0,0,1)'], legend_text='High')

HSESH_rho_epsilon125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.25], x_var='rho', y_var='H_Share_L', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=True, colors=['rgba(0,0,200,1)', 'rgba(0,0,200,1)'], legend_text='Low')


HSESH_rho_epsilon125.data[3]['line']['dash'] = 'dash'
HSESL_rho_epsilon125.data[3]['line']['dash'] = 'dash'

fig_HSES_rho = go.Figure(data = HSESH_rho_epsilon125.data + HSESL_rho_epsilon125.data)

fig_HSES_rho.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'HSES Share',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.5),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.25,0.75],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_HSES_rho.add_hline(y=0.5, line_width=2, annotation_text = '', annotation_font_color='black', line=dict(color='black'))
fig_HSES_rho.add_vline(x=1, line_width=2, annotation_text = 'ρ=1: no bias', annotation_font_color='black', line=dict(color='black'))

fig_HSES_rho.data[0]['name'] = 'Realised  L'
fig_HSES_rho.data[3]['name'] = 'Predicted L'
fig_HSES_rho.data[6]['name'] = 'Realised  H'
fig_HSES_rho.data[9]['name'] = 'Predicted H'
fig_HSES_rho


### Rho Assortativity

In [65]:

SESAss_rho_epsilon1 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1], x_var='rho', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(250,0,250,1)'], legend_text='ϵ=1')

SESAss_rho_epsilon125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.25], x_var='rho', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(150,0,150,1)'], legend_text='ϵ=1.25')

SESAss_rho_epsilon15 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.5], x_var='rho', y_var='Network_SES_assortativity', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(100,0,100,1)'], legend_text='ϵ=1.5')


SESAss_rho_epsilon125 = go.Figure(data = SESAss_rho_epsilon1.data + SESAss_rho_epsilon125.data + SESAss_rho_epsilon15.data)

SESAss_rho_epsilon125.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'SES Assortativity',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.15),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [-0.5,0.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


SESAss_rho_epsilon125.add_hline(y=0, line_width=2, annotation_text = 'Random sorting', annotation_font_color='black', line=dict(color='black'))
SESAss_rho_epsilon125.add_vline(x=1, line_width=2, annotation_text = 'ρ=1: no bias', annotation_font_color='black', line=dict(color='black'))




### Rho Exposure

In [106]:

Exposure_rho_epsilon1 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1], x_var='rho', y_var='Exposure', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(15,209,132,1)'], legend_text='ϵ=1')

Exposure_rho_epsilon125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.25], x_var='rho', y_var='Exposure', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(15,170,100,1)'], legend_text='ϵ=1.25')

Exposure_rho_epsilon15 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.5], x_var='rho', y_var='Exposure', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=False, colors=['rgba(15,100,50,1)'], legend_text='ϵ=1.5')


fig_Exposure_rho = go.Figure(data = Exposure_rho_epsilon15.data + Exposure_rho_epsilon125.data + Exposure_rho_epsilon1.data)

fig_Exposure_rho.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Rho', 
    yaxis_title = 'Exposure',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.95),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,1.5],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_Exposure_rho.add_hline(y=1, line_width=2, annotation_text = 'Even exposure', annotation_font_color='black', line=dict(color='black'))
fig_Exposure_rho.add_vline(x=1, line_width=2, annotation_text = 'ρ=1: no bias', annotation_font_color='black', line=dict(color='black'))




### Rho Friending

In [107]:



Friending_rho_epsilon125 = plotly_lines.plot_MTO_sim_results(plotly_lines(), data=df_rho[df_rho['epsilon'] == 1.25], x_var='rho', y_var='Friend_Bias', x_range=[0.5,2], y_range=[0,1], x_title='rho', y_title='Exposure', with_predicted=True, colors=['rgba(229,187,0,1)', 'rgba(224,116,0,1)'], legend_text='High')


fig_Friending_rho = go.Figure(data = Friending_rho_epsilon125.data)

fig_Friending_rho.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'rho', 
    yaxis_title = 'Friending Effect',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.05),
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [-0.4,0.4],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')

fig_Friending_rho.data[0]['line']['width'] = 3
fig_Friending_rho.data[3]['line']['dash'] = 'dash'

fig_Friending_rho.add_hline(y=0, line_width=2, annotation_text = 'Neutral Friending', annotation_font_color='black', line=dict(color='black'))
fig_Friending_rho.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no bias', annotation_font_color='black', line=dict(color='black'))

fig_Friending_rho.add_shape(
    type="line",
    x0=0, y0=0,
    x1=1, y1=0,
    line=dict(color='rgba(229,187,0,1)', width=3)
)



## Both rho and epsilon

### Specified values

In [2]:

# general simulation stuff
list_eps_values = np.arange(0.5, 2, 0.05)
list_rho_values = np.arange(0.5, 2, 0.05)

num_sim_per_param = 5 # number of simulations with given parameter set
num_MTO_per_sim = 50 # number of simulations on each network

#df_eps_rho = MTO_simulator(list_eps_values=list_eps_values, list_rho_values=list_rho_values, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_eps_rho_2 = MTO_simulator(list_eps_values=list_eps_values, list_rho_values=list_rho_values_2, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_eps_rho.to_csv('eps_rho_sim_new.csv')

df_eps_rho = pd.read_csv('SimulationData/eps_rho_sim_new.csv')
#df_eps_rho = pd.read_csv('eps_rho_sim.csv')

df_eps_rho


,Unnamed: 0,sim_id,p_SES_high,epsilon,rho,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,Network_H_Share_H,Network_H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias
0,0,ViskWJtc,0.5,0.50,0.50,0.894737,1.789474,0.000000,20,19,-0.264149,0.398503,0.677811,31.94,0.360073,1.333333,1.333333,-0.200000
1,1,ViskWJtc,0.5,0.50,0.50,0.500000,1.000000,0.000000,20,18,-0.264149,0.398503,0.677811,31.94,0.360073,1.333333,1.333333,-0.200000
2,2,ViskWJtc,0.5,0.50,0.50,0.736842,1.473684,0.000000,20,19,-0.264149,0.398503,0.677811,31.94,0.360073,1.333333,1.333333,-0.200000
3,3,ViskWJtc,0.5,0.50,0.50,0.833333,1.666667,0.000000,20,18,-0.264149,0.398503,0.677811,31.94,0.360073,1.333333,1.333333,-0.200000
4,4,ViskWJtc,0.5,0.50,0.50,0.500000,1.000000,0.000000,20,20,-0.264149,0.398503,0.677811,31.94,0.360073,1.333333,1.333333,-0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224995,224995,uaXTRqJt,0.5,1.95,1.95,0.300000,0.923077,0.350000,14,10,0.522975,0.775261,0.236330,20.22,0.281087,0.173837,0.677966,0.385737
224996,224996,uaXTRqJt,0.5,1.95,1.95,0.083333,0.307692,0.458333,14,12,0.522975,0.775261,0.236330,20.22,0.281087,0.173837,0.677966,0.385737
224997,224997,uaXTRqJt,0.5,1.95,1.95,0.083333,0.428571,0.611111,14,12,0.522975,0.775261,0.236330,20.22,0.281087,0.173837,0.677966,0.385737
224998,224998,uaXTRqJt,0.5,1.95,1.95,0.153846,0.428571,0.282051,14,13,0.522975,0.775261,0.236330,20.22,0.281087,0.173837,0.677966,0.385737


### Share of H friends

In [ ]:

fig_HSES_H = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Network_H_Share_H'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.diverging.balance, range_color=(0,1))

fig_HSES_H.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 600, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Network<br>HSES Share<br>of HSES',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_HSES_H.add_hline(y=1, line_width=2, annotation_text = 'ρ=1: no friending bias', annotation_font_color='black', line=dict(color='black'))
fig_HSES_H.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no exposure bias', annotation_font_color='black', line=dict(color='black'))


In [ ]:

fig_HSES_L = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Network_H_Share_L'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.diverging.balance, range_color=(0,1))

fig_HSES_L.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 600, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Network<br>HSES Share<br>of LSES',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_HSES_L.add_hline(y=1, line_width=2, annotation_text = 'ρ=1: no friending bias', annotation_font_color='black', line=dict(color='black'))
fig_HSES_L.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no exposure bias', annotation_font_color='black', line=dict(color='black'))


### SES Assortativity

In [ ]:

fig_SESAss = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Network_SES_assortativity'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.diverging.PuOr_r, range_color=(-0.5,0.5))

fig_SESAss.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 600, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'SES<br>Assortativity',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_SESAss.add_hline(y=1, line_width=2, annotation_text = 'ρ=1: no friending bias', annotation_font_color='black', line=dict(color='black'))
fig_SESAss.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no exposure bias', annotation_font_color='black', line=dict(color='black'))


### Degree and clustering (appendix)

In [ ]:


fig_friending = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Degree'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.sequential.Plotly3)

fig_friending.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 600, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Mean<br>Realised<br>Friending',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_friending.add_hline(y=1, line_width=2, annotation_text = 'ρ=1: no friending bias', annotation_font_color='black', line=dict(color='black'))
fig_friending.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no exposure bias', annotation_font_color='black', line=dict(color='black'))




In [ ]:


fig_friending = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Network_Clustering'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.sequential.Plotly3)

fig_friending.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 600, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Mean<br>Realised<br>Friending',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_friending.add_hline(y=1, line_width=2, annotation_text = 'ρ=1: no friending bias', annotation_font_color='black', line=dict(color='black'))
fig_friending.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no exposure bias', annotation_font_color='black', line=dict(color='black'))




### Exposure effect

In [ ]:


fig_pred_exposure = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Predicted_Exposure'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.diverging.Fall_r, range_color=(0.6,1.4))

fig_pred_exposure.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 600, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Predicted<br>Exposure',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_pred_exposure.add_hline(y=1, line_width=2, annotation_text = 'ρ=1: no friending bias', annotation_font_color='black', line=dict(color='black'))
fig_pred_exposure.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no exposure bias', annotation_font_color='black', line=dict(color='black'))




In [6]:

fig_exposure = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Exposure'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.diverging.Fall_r, range_color=(0.6,1.4))

fig_exposure.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 600, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Friending Bias'),
    coloraxis_colorbar_title_text = 'Mean<br>Realised<br>Exposure',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_exposure.add_hline(y=1, line_width=2, annotation_text = 'ρ=1: no friending bias  ', annotation_font_size=20, annotation_font_color="black", line=dict(color='black'))
fig_exposure.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no exposure bias   ', annotation_font_size=20, annotation_font_color="black", line=dict(color='black'))


ValueError: 
    Invalid value of type 'builtins.str' received for the 'histfunc' property of histogram2d
        Received value: 'std'

    The 'histfunc' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['count', 'sum', 'avg', 'min', 'max']

### Friending Effect

In [ ]:


fig_pred_friending = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Predicted_Friend_Bias'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.diverging.Earth_r, range_color=(-0.5,0.5))

fig_pred_friending.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 600, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Predicted Friending'),
    coloraxis_colorbar_title_text = 'Predicted<br>Friending',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_pred_friending.add_hline(y=1, line_width=2, annotation_text = 'ρ=1: no friending bias', annotation_font_color='black', line=dict(color='black'))
fig_pred_friending.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no exposure bias', annotation_font_color='black', line=dict(color='black'))




In [ ]:


fig_friending = px.density_heatmap(x=df_eps_rho['epsilon'], y=df_eps_rho['rho'], z=df_eps_rho['Friend_Bias'], range_x=[0.5,2], range_y=[1,2], histfunc='avg', color_continuous_scale=px.colors.diverging.Earth_r, range_color=(-0.5,0.5))

fig_friending.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Epsilon', 
    yaxis_title = 'Rho',
    width = 600, height = 400,
    showlegend = True, 
    legend=dict(x=0.75,y=1, title='Mean Exposure'),
    coloraxis_colorbar_title_text = 'Mean<br>Realised<br>Friending',
    legend_title_text = '',
    xaxis_range = [0.5,2],
    yaxis_range = [0.5,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_friending.add_hline(y=1, line_width=2, annotation_text = 'ρ=1: no friending bias', annotation_font_color='black', line=dict(color='black'))
fig_friending.add_vline(x=1, line_width=2, annotation_text = 'ϵ=1: no exposure bias', annotation_font_color='black', line=dict(color='black'))




## Chetty values

In [4]:
#%%capture

# get the Chetty epsilon and rho values: 3012 valid counties
list_chetty_eps = list(df_eps_rho_chetty['Epsilon'])
list_chetty_rho = list(df_eps_rho_chetty['Rho'])

#df_rho_eps = MTO_simulator(list_eps_values=list_eps_values_random, list_rho_values=list_rho_values_random, num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
#df_rho_eps.sample(5)


num_sim_per_param = 5 # number of simulations with given parameter set
num_MTO_per_sim = 10 # number of simulations on each network

df_chetty_sims = []
'''
for i in range(len(list_chetty_eps)):

    print('\r', f'{i}', end='')

    df_eps_rho_chetty_sim = MTO_simulator(list_eps_values=[list_chetty_eps[i]], list_rho_values=[list_chetty_rho[i]], num_MTO_per_sim=num_MTO_per_sim, num_sim_per_param=num_sim_per_param)
    df_chetty_sims.append(df_eps_rho_chetty_sim)


df_chetty_sims_final = pd.concat(df_chetty_sims)
'''

#df_chetty_sims_final.to_csv('chettysims_new.csv')

df_chetty_sims_final = pd.read_csv('SimulationData/chettysims_new.csv')


### Exposure pred vs real

In [ ]:

df_test = df_chetty_sims_final.groupby('sim_id', as_index=False).mean().rename(columns={'epsilon': 'Epsilon', 'rho': 'Rho'})
df_test = pd.merge(left = df_test, right = df_exp_fe_chetty, on=['Epsilon', 'Rho'])

fig_expo = px.scatter(x=df_test['exposure_grp_mem_county'], y=df_test['Exposure'], range_x=[0, 2], range_y=[0, 2], color=df_test['Epsilon'], color_continuous_scale=px.colors.diverging.Fall, range_color=(0.5, 1.5), color_continuous_midpoint=1)

fig_expo.add_shape(
    type="line",
    x0=-2, y0=-2,
    x1=2, y1=2,
    line=dict(color="black", width=3, dash="dash")
)

fig_expo.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Observed Exposure Effect', 
    yaxis_title = 'Simulation Exposure Effect',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.65),
    legend_title_text = '',
    coloraxis_colorbar_title_text = 'ϵ',
    xaxis_range = [0,2],
    yaxis_range = [0,2],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


fig_expo.add_hline(y=1, line_width=2, annotation_font_color='black', line=dict(color='black'), annotation_text='')
fig_expo.add_vline(x=1, line_width=2, annotation_font_color='black', line=dict(color='black'), annotation_text='')



In [11]:

result = sm.ols(formula="Exposure ~ Predicted_Exposure", data=df_chetty_sims_final).fit()
print(np.corrcoef(df_chetty_sims_final['Exposure'], df_chetty_sims_final['Predicted_Exposure']))
print(result.summary())

#px.scatter(df_rho_eps, x='Predicted_Friend_Bias', y='Friend_Bias', height=400, width=400)


[[1.         0.60800101]
 [0.60800101 1.        ]]
                            OLS Regression Results                            
Dep. Variable:               Exposure   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                 8.832e+04
Date:                Sun, 07 May 2023   Prob (F-statistic):               0.00
Time:                        14:57:23   Log-Likelihood:                -10579.
No. Observations:              150600   AIC:                         2.116e+04
Df Residuals:                  150598   BIC:                         2.118e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------

### Friending Pred vs Real

In [ ]:

fig_fe = px.scatter(x=df_test['bias_grp_mem_county'], y=df_test['Friend_Bias'], range_x=[0, 2], range_y=[0, 2], color=df_test['Rho'], color_continuous_scale=px.colors.diverging.balance_r, range_color=(0.5,1.5), color_continuous_midpoint=1)

fig_fe.add_shape(
    type="line",
    x0=-2, y0=-2,
    x1=2, y1=2,
    line=dict(color="black", width=3, dash="dash")
)

fig_fe.update_layout(
    title = None, 
    font=dict(size = 18),
    xaxis_title = 'Observed Friending Effect', 
    yaxis_title = 'Simulation Friending Effect',
    width = 500, height = 500,
    showlegend = True, 
    legend=dict(x=0.75,y=0.65),
    coloraxis_colorbar_title_text = 'ρ',
    legend_title_text = '',
    xaxis_range = [-0.1,0.3],
    yaxis_range = [-0.1,0.3],
    margin=dict(b=5,l=5,r=5,t=5),
    xaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2), 
    yaxis = dict(showgrid = False, showline = True, linecolor = 'rgb(0,0,0)', linewidth = 2),
    plot_bgcolor='rgba(0,0,0,0)')


#fig_fe.add_hline(y=0, line_width=2, annotation_font_color='black', line=dict(color='black'), annotation_text='')
fig_fe.add_vline(x=0, line_width=2, annotation_font_color='black', line=dict(color='black'), annotation_text='')

fig_fe.add_shape(
    type="line",
    x0=0, y0=0,
    x1=2, y1=0,
    line=dict(color="black", width=2)
)


In [10]:

result = sm.ols(formula="Friend_Bias ~ Predicted_Friend_Bias", data=df_chetty_sims_final).fit()
print(np.corrcoef(df_chetty_sims_final['Friend_Bias'], df_chetty_sims_final['Predicted_Friend_Bias']))
print(result.summary())



#px.scatter(df_rho_eps, x='Predicted_Friend_Bias', y='Friend_Bias', height=400, width=400)


[[1.         0.41803504]
 [0.41803504 1.        ]]
                            OLS Regression Results                            
Dep. Variable:            Friend_Bias   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                  0.175
Method:                 Least Squares   F-statistic:                 3.189e+04
Date:                Sun, 07 May 2023   Prob (F-statistic):               0.00
Time:                        14:57:05   Log-Likelihood:             1.3098e+05
No. Observations:              150600   AIC:                        -2.620e+05
Df Residuals:                  150598   BIC:                        -2.619e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------

### Descriptive statistics

### Regressions

In [13]:

df_chetty_sims_final.groupby(['epsilon', 'rho']).var()
#df_test = df_results[['Predicted_H_Share', 'H_Share', 'Predicted_Exposure', 'Exposure', 'Predicted_Friend_Bias', 'Friend_Bias', 'Degree', 'N_exposure']].describe()



,,Unnamed: 0,p_SES_high,H_Share,Exposure,Friend_Bias,N_exposure,Degree,Network_SES_assortativity,Network_H_Share_H,Network_H_Share_L,Network_Degree,Network_Clustering,Predicted_H_Share,Predicted_Exposure,Predicted_Friend_Bias
epsilon,rho,,,,,,,,,,,,,,,
0.345641,1.501423,212.5,0.0,0.018006,0.057057,0.008213,5.877551,5.387755,0.000338,0.005273,0.005132,5.640229,0.000617,0.0,0.0,0.0
0.366055,1.127154,212.5,0.0,0.016700,0.049994,0.004625,5.877551,6.303673,0.000719,0.001366,0.002112,8.983249,0.000710,0.0,0.0,0.0
0.384495,1.272278,212.5,0.0,0.019290,0.064917,0.004096,5.877551,4.702449,0.001117,0.005950,0.006621,8.051363,0.000901,0.0,0.0,0.0
0.395897,1.098400,212.5,0.0,0.015889,0.051446,0.003517,5.877551,4.931020,0.000477,0.010559,0.009507,8.477616,0.000786,0.0,0.0,0.0
0.396414,0.895840,212.5,0.0,0.010858,0.043434,0.000000,5.877551,5.275510,0.000284,0.001799,0.001087,10.283984,0.000968,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5.009796,1.085526,212.5,0.0,0.008524,0.037188,0.036253,5.877551,5.520408,0.000625,0.000430,0.000684,8.596735,0.001273,0.0,0.0,0.0
5.217939,1.352737,212.5,0.0,0.007087,0.031687,0.101663,5.877551,5.551429,0.000233,0.001044,0.001026,7.564833,0.001327,0.0,0.0,0.0
6.101012,1.585198,212.5,0.0,0.005282,0.025310,0.101567,5.877551,3.275510,0.000121,0.000020,0.000213,7.156180,0.000889,0.0,0.0,0.0


In [18]:

result = sm.ols(formula="Exposure ~ Predicted_Exposure + epsilon + rho", data=df_chetty_sims_final).fit()
print(result.params)
print(result.summary())


Intercept             0.130417
Predicted_Exposure    0.883162
epsilon              -0.018637
rho                   0.004173
dtype: float64
                            OLS Regression Results                            
Dep. Variable:               Exposure   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.370
Method:                 Least Squares   F-statistic:                 2.946e+04
Date:                Sun, 07 May 2023   Prob (F-statistic):               0.00
Time:                        14:59:41   Log-Likelihood:                -10556.
No. Observations:              150600   AIC:                         2.112e+04
Df Residuals:                  150596   BIC:                         2.116e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.

In [17]:

result = sm.ols(formula="Friend_Bias ~ Predicted_Friend_Bias + epsilon + rho", data=df_chetty_sims_final).fit()
print(result.params)
print(result.summary())


Intercept               -0.034334
Predicted_Friend_Bias    0.828448
epsilon                  0.000413
rho                      0.043488
dtype: float64
                            OLS Regression Results                            
Dep. Variable:            Friend_Bias   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                  0.175
Method:                 Least Squares   F-statistic:                 1.064e+04
Date:                Sun, 07 May 2023   Prob (F-statistic):               0.00
Time:                        14:59:26   Log-Likelihood:             1.3099e+05
No. Observations:              150600   AIC:                        -2.620e+05
Df Residuals:                  150596   BIC:                        -2.619e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t     